In [ ]:
import os
import gc
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import tensorflow as tf
from tqdm.autonotebook import tqdm

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

from keras import Sequential
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam, SGD
from keras.callbacks import ReduceLROnPlateau
from keras.layers import Flatten, Dense, BatchNormalization, Activation, Dropout
from keras.layers import Lambda, Input, GlobalAveragePooling2D, BatchNormalization

from tensorflow.keras.models import Model

from keras.preprocessing.image import load_img

In [ ]:
label = pd.read_csv('labels.csv')
label.head()

In [ ]:
label.describe()

In [ ]:
def cls(a):
    for i in a.patches:
        val = i.get_width()
        x = i.get_x() + i.get_width()
        y = i.get_y()+ i.get_height()/2
        a.annotate(round(val,2), (x,y))
plt.figure(figsize = (20, 40))
a1 = sns.countplot(y=label['breed'], order = label['breed'].value_counts().index)
cls(a1)
plt.show()

In [ ]:
specis = sorted(list(set(label['breed'])))
n = len(specis)
print("Total breed {}".format(n))
d = dict(zip(specis, range(n)))
d

In [ ]:
input_shape = (331,331,3)

def img_arr(directory, label_dataframe, target_size = input_shape):
    
    img_label = label_dataframe['breed']
    images = np.zeros([len(label_dataframe), target_size[0], target_size[1], target_size[2]], dtype = np.uint8)
    y = np.zeros([len(label_dataframe), 1],dtype = np.uint8)
    for i, name in enumerate(tqdm(label_dataframe['id'].values)):
        img_dir = os.path.join(directory, name + '.jpg')
        img = load_img(img_dir, target_size = target_size)
        images[i] = img
        del img
        dog_breed = img_label[i]
        y[i] = d[dog_breed]
    y = to_categorical(y)
    return images, y

In [ ]:
import os
if len(os.listdir('C:/Users/Debasmita/OneDrive/Desktop/New folder/train/')) == len(label['id']):
    print('Number of file matches number of actual images!')
else:
    print('Number of file doesnot matches number of actual images!!')

In [ ]:
import time 
t = time.time()

X,y = img_arr('C:/Users/Debasmita/OneDrive/Desktop/New folder/train/', label[:])

print('runtime in seconds: {}'.format(time.time() - t))

In [ ]:
p=25

# setup the figure
plt.figure(figsize=(20,20))

for i in range(p):
    a = plt.subplot(5, 5, i+1)
    plt.title(specis[np.where(y[i] == 1)[0][0]])
    plt.imshow(X[i].astype('int32'))

In [ ]:
lrr= ReduceLROnPlateau(monitor='val_accuracy', factor=.01, patience=3, min_lr=1e-5,verbose = 1)

#Prepare call backs
EarlyStop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [ ]:
batch_size= 128
epochs=50
learn_rate=.001
sgd=SGD(lr=learn_rate,momentum=.9,nesterov=False)
adam=Adam(lr=learn_rate, beta_1=0.9, beta_2=0.999, epsilon=None,  amsgrad=False)

In [ ]:
image_size=(224,224)

In [ ]:
img_size = (331,331,3)

def get_features(model_name, model_preprocessor, input_size, data):

    input_layer = Input(input_size)
    preprocessor = Lambda(model_preprocessor)(input_layer)
    base_model = model_name(weights='imagenet', include_top=False,
                            input_shape=input_size)(preprocessor)
    avg = GlobalAveragePooling2D()(base_model)
    feature_extractor = Model(inputs = input_layer, outputs = avg)
    
    #Extract feature.
    feature_maps = feature_extractor.predict(data, verbose=1)
    print('Feature maps shape: ', feature_maps.shape)
    return feature_maps

In [ ]:
from keras.applications.inception_v3 import InceptionV3, preprocess_input
inception_preprocessor = preprocess_input
inception_features = get_features(InceptionV3,
                                  inception_preprocessor,
                                  img_size, X)

In [ ]:
from keras.applications.xception import Xception, preprocess_input
xception_preprocessor = preprocess_input
xception_features = get_features(Xception,
                                 xception_preprocessor,
                                 img_size, X)

In [ ]:
from keras.applications.inception_resnet_v2 import InceptionResNetV2, preprocess_input
inc_resnet_preprocessor = preprocess_input
inc_resnet_features = get_features(InceptionResNetV2,
                                   inc_resnet_preprocessor,
                                   img_size, X)

In [ ]:
from keras.applications.nasnet import NASNetLarge, preprocess_input
nasnet_preprocessor = preprocess_input
nasnet_features = get_features(NASNetLarge,
                               nasnet_preprocessor,
                               img_size, X)

In [ ]:
del X #to free up some ram memory
gc.collect()

In [ ]:
final_features = np.concatenate([inception_features,
                                 xception_features,
                                 nasnet_features,
                                 inc_resnet_features,], axis=-1) #axis=-1 to concatinate horizontally

print('Final feature maps shape', final_features.shape)

In [ ]:
model = Sequential() 
model.add(Dropout(0.7,input_shape=(final_features.shape[1],)))
model.add(Dense(n,activation= 'softmax'))

model.compile(optimizer=adam,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

#Training the model. 
history = model.fit(final_features, y,
            batch_size=batch_size,
            epochs=epochs,
            validation_split=0.2,
            callbacks=[lrr,EarlyStop])

In [ ]:
history_df = pd.DataFrame(history.history)
history_df.loc[:, ['loss', 'val_loss']].plot();
history_df.loc[:, ['accuracy', 'val_accuracy']].plot();

In [ ]:
del inception_features
del xception_features
del nasnet_features
del inc_resnet_features
del final_features
gc.collect()

In [ ]:
sample = pd.read_csv('sample_submission.csv')

In [ ]:
sample.shape

In [ ]:
sample.describe()

In [ ]:
def images_to_array_test(test_path, img_size = (331,331,3)):
    test_filenames = [test_path + fname for fname in os.listdir(test_path)]

    data_size = len(test_filenames)
    images = np.zeros([data_size, img_size[0], img_size[1], 3], dtype=np.uint8)
    
    
    for ix,img_dir in enumerate(tqdm(test_filenames)):
        img = load_img(img_dir, target_size = img_size)
        images[ix]=img
        del img
    print('Ouptut Data Size: ', images.shape)
    return images

test_data = images_to_array_test('C:/Users/Debasmita/OneDrive/Desktop/New folder/test/', img_size)

In [ ]:
def extact_features(data):
    inception_features = get_features(InceptionV3, inception_preprocessor, img_size, data)
    xception_features = get_features(Xception, xception_preprocessor, img_size, data)
    nasnet_features = get_features(NASNetLarge, nasnet_preprocessor, img_size, data)
    inc_resnet_features = get_features(InceptionResNetV2, inc_resnet_preprocessor, img_size, data)

    final_features = np.concatenate([inception_features,
                                     xception_features,
                                     nasnet_features,
                                     inc_resnet_features],axis=-1)
    
    print('Final feature maps shape', final_features.shape)
    
    #deleting to free up ram memory
    del inception_features
    del xception_features
    del nasnet_features
    del inc_resnet_features
    gc.collect()

In [ ]:
del test_data
gc.collect()

In [ ]:
pred = model.predict(test_features)

In [ ]:
print(pred[0])
print(f"Max value (probability of prediction): {np.max(pred[0])}") # the max probability value predicted by the model
print(f"Sum: {np.sum(pred[0])}") # because we used softmax activation in our model, this will be close to 1
print(f"Max index: {np.argmax(pred[0])}") # the index of where the max value in predictions[0] occurs
print(f"Predicted label: {specis[np.argmax(pred[0])]}")

In [ ]:
preds = pd.DataFrame(columns=["id"] + list(specis))
preds.head()

In [ ]:
test_path = "C:/Users/Debasmita/OneDrive/Desktop/New folder/test"
preds["id"] = [os.path.splitext(path)[0] for path in os.listdir(test_path)]
preds.head()

In [ ]:
preds.loc[:,list(specis)]= pred

preds.to_csv('submission.csv',index=None)
preds.head()